In [1]:
#export
import os
import time
import atexit
import argparse
import datetime
from pathlib import Path
from functools import partial

import numpy  as np
import GPUtil as gu

# Code

In [5]:
class GpuLockedTimeout(Exception):pass
class GpuUsageTimeout(Exception):pass

In [2]:
#export
class Lock:
    def __init__(self, path, data, seconds_delay):
        timedelta = datetime.timedelta(seconds=seconds_delay)
        time_out = datetime.datetime.now() + timedelta
        timestamp = '{:%Y_%b_%d_%H_%M_%S_%f}'.format(time_out)
        lock_name = f'dhpo_{timestamp}.lock'
        self.lock_name = path / lock_name
        self.data = data   
    
    def __enter__(self):
        try:
            print(f'Locking "{self.data}" @ {self.lock_name}')
            with open(self.lock_name, 'w+') as f:
                f.write(self.data)
            os.chmod(self.lock_name, 0o776)
        except Exception as e:
            print(f'Cant get a lock! {self.lock_name} \n', e)
            self.lock_name = None
            
        return self.lock_name
    
    def __exit__(self, type, value, traceback):
        try:
            if self.lock_name:
                os.remove(self.lock_name)
        except Exception as e:
            print(f'Cant remove lock! {self.lock_name}\n', e)


In [3]:
#export
def base_exit_handler(lock_name):
    try:
        if lock_name:
            if os.path.exists(lock_name): os.remove(lock_name)
    except Exception as e:
        print(f'Cant remove lock! {lock_name}\n', e)

def lock(gpus, seconds_delay, path='/tmp'):
    gpus = str(gpus).strip('()')
    lock = Lock(path, gpus, seconds_delay)
    with lock as l:
        exit_handler = partial(base_exit_handler, lock_name=l)
        atexit.register(exit_handler)
        time.sleep(seconds_delay)

In [ ]:
#export
def check_locks(path):
    g = path.rglob('dhpo_*.lock')
    locked_gpus = set()
    for l in g:
        if is_expired(l):
            print(f'Lock {l} is expired, deleting')
            clean_up(l)
            continue
        locked_gpus.update(read_lock(l))
    return locked_gpus

def list_locks(path):
    g = path.rglob('dhpo_*.lock')
    if g:
        for i, l in enumerate(g):
            print(f'\t{i}. {l}')
            with open(l, 'r') as f:
                print(f'\t\tGPUS: {f.read()}')
    else:
        print('\tThere is no locks')
    
def is_expired(name):
    date = name.name.strip('dhpo_').rstrip('.lock')
    datetime_object = datetime.datetime.strptime(date, '%Y_%b_%d_%H_%M_%S_%f')
    datetime_object, datetime_object > datetime.datetime.now()
    return datetime_object < datetime.datetime.now()

def clean_up(name):
    print(f'\n\t ooo deleting{name}')
    os.remove(name)
    
def read_lock(name):
    with open(name, 'r') as f:
        data = f.read()
    return set([int(i) for i in data if i.isdigit()])

In [10]:
#export
def check_gpu_access(path, gpus):
    print(f'\n\tChecking {path} for locks on gpu {gpus}...')
    #path = Path('./test')
    #gpus='0'
    #lock_delay = 5

    target_gpu = set([int(i) for i in gpus if i.isdigit()])
    all_gpus = set((0,1,2,3))

    #while True:
    locked_gpus = check_locks(path)
    avail_gpus = all_gpus - locked_gpus    
    if target_gpu.issubset(avail_gpus):
        print(f'\n\tgpu {target_gpu} is avaliable...')
        #break
    else:
        print(f'\n\tWaiting, gpu #{target_gpu} is locked. Current gpus under lock: {locked_gpus}')
        #time.sleep(lock_delay)
        raise GpuLockedTimeout

In [8]:
#export
def check_gpu_usage(gpus, threshlod=.2, delay=.1):
    gpus = set([int(i) for i in gpus if i.isdigit()])
    load = get_gpu_load(delay)
    for g in gpus:
        if load[g] > threshlod:
            print(f'Usage on gpu{g} : {load[g]}')
            raise GpuUsageTimeout
    print(f'Gpu usage is acceptable: {load}')
    
def get_gpu_load(delay=.1):
    load = []
    for i in range(10):
        gpus = gu.getGPUs()
        c_load = [g.load for g in gpus]
        load.append(c_load)
        time.sleep(delay)
    return np.array(load).mean(axis=0)

In [17]:
#check_gpu_access('0')

In [5]:
#export
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--mdelay', '-md', dest='minute_delay',
                    default=.1,
                    help='delay in minutes')
    parser.add_argument('--gpus', '-g', dest='gpus',
                    default=None,
                    help='gpus to lock, 0,2')
    
    args = parser.parse_args()
    gpus = args.gpus
    p = Path('/home/sokolov/work/cycler/dHPO/sync/locks')
    
    minute_delay = args.minute_delay
    seconds_delay = float(minute_delay) * 60
    print(f'DELAY: {seconds_delay} seconds')
    print(f'GPUS: {gpus}')
    print(f'PATH: {p}')
    lock(path=p, gpus=gpus, seconds_delay=seconds_delay)
    

usage: ipykernel_launcher.py [-h] [--mdelay MINUTE_DELAY] [--gpus GPUS]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/sokolov/.local/share/jupyter/runtime/kernel-e361d33f-8a7a-4c98-9260-9c30f637b817.json


SystemExit: 2

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Tests

In [6]:
lock(gpus='0,1', seconds_delay=10)

Locking "0,1" @ test/dhpo_2020_May_13_18_02_56_001469.lock


In [ ]:
#! rm ./test/*

# Exports

In [19]:
!python3 extra/n2s.py locker.ipynb

Converted locker.ipynb to exp/nb_locker.py


In [23]:
!python3 exp/nb_locker.py --gpus=0,1 --mdelay=.05

DELAY: 3.0 seconds
GPUS: 0,1
PATH: /home/sokolov/work/cycler/crsch_cycle/locks
Locking "0,1" @ /home/sokolov/work/cycler/crsch_cycle/locks/dhpo_2020_May_13_20_35_07_988504.lock
